In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score as R2
from sklearn.metrics import mean_squared_error as MSE

In [2]:
rest = pd.read_csv('/Volumes/Database/Research/ozone-budget/result_df_UKESM1_3_0_NN_width_64_dropout_0.1_90.csv')
data = pd.read_csv('/Volumes/Database/Research/C7 Multivariate analysis/UKESM1-0-LL_diag_2014(1217).csv')

In [6]:
list(rest)

['HO₂',
 'CH₃O₂',
 'kₓROₓ',
 'O¹D',
 'kₐalkenes',
 'PO₃̂',
 'LO₃̂',
 'longitude',
 'latitude',
 't',
 'o3',
 'no2',
 'no',
 'oh',
 'h2o',
 'hno3',
 'o3prod',
 'o3loss',
 'k_no_ho2',
 'k_no_ch3o2',
 'k_o1d_h2o',
 'k_o3_oh',
 'k_o3_ho2',
 'tas',
 'pan',
 'photo1d',
 'o3_mole',
 'no_mole',
 'no2_mole',
 'oh_mole',
 'h2o_mole',
 'hno3_mole',
 'pan_mole',
 'ho2_mole',
 'ch3o2_mole',
 'k_ho2_ho2',
 'k_oh_no2',
 'Pt_HO2_NO',
 'Pt_CH3O2_NO',
 'Pt_HO2_O3',
 'Pt_OH_O3']

In [34]:
rest['tas'] = data['tas']
rest['pan'] = data['pan']
rest['photo1d'] = data['photo1d']

rest['o3_mole'] = rest['o3']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['no_mole'] = rest['no']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['no2_mole'] = rest['no2']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['oh_mole'] = rest['oh']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['h2o_mole'] = rest['h2o']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['hno3_mole'] = rest['hno3']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['pan_mole'] = rest['pan']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)

rest['ho2_mole'] = rest['HO₂']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['ch3o2_mole'] = rest['CH₃O₂']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['o1d_mole'] = rest['O¹D']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)

rest['k_ho2_ho2'] = 2.2*10**(-13)*np.exp(600/rest['tas'])
rest['k_oh_no2'] = 6.5*10**(-11)
rest['k_no_ho2'] = data['k_no_ho2']
rest['k_no_ch3o2'] = data['k_no_ch3o2']
rest['k_o1d_h2o'] = 2.14*10**(-10)
rest['k_o3_ho2'] = data['k_o3_ho2']
rest['k_o3_oh'] = data['k_o3_oh']

In [39]:
rest['Pt_HO2_NO'] = rest['k_no_ho2']*rest['no_mole']*rest['ho2_mole']
rest['Pt_CH3O2_NO'] = rest['k_no_ch3o2']*rest['no_mole']*rest['ch3o2_mole']
rest['Pt_RO2_NO'] = rest['PO₃̂']-rest['Pt_HO2_NO']-rest['Pt_CH3O2_NO']

rest['Pt_O1D_H2O'] = rest['k_o1d_h2o']*rest['o1d_mole']*rest['h2o_mole']*rest['photo1d']*rest['o3_m']
rest['Pt_HO2_O3'] = rest['k_o3_ho2']*rest['ho2_mole']*rest['o3_mole']
rest['Pt_OH_O3'] = rest['k_o3_oh']*rest['oh_mole']*rest['o3_mole']
rest['Pt_VOCs_O3'] = rest['LO₃̂']-rest['Pt_O1D_H2O']-rest['Pt_HO2_O3']-rest['Pt_OH_O3']

In [40]:
country = pd.read_csv('/Users/csuen/Desktop/PhD/Maps/country_province_state_2*2.csv')
rest = rest.merge(country, how = 'left', left_on = ['longitude','latitude'], right_on = ['longitude','latitude'])
rest = rest.drop(['province','state'], axis=1)
rest_land = rest.dropna()

In [41]:
rest_land['PO₃̂'].mean(), rest_land['Pt_HO2_NO'].mean(), rest_land['Pt_CH3O2_NO'].mean(), rest_land['Pt_RO2_NO'].mean()

(2771131.4303185223,
 1401355.8334046982,
 1331463.2983078114,
 38312.298606030454)

In [42]:
rest_land['LO₃̂'].mean(), rest_land['Pt_O1D_H2O'].mean(), rest_land['Pt_HO2_O3'].mean(), rest_land['Pt_OH_O3'].mean()

(688458.8364923124,
 5.526838546902233e-12,
 2325.715233444886,
 67343.61033307547)